##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Готовые оценщики

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/estimator/premade"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Смотреть на TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ru/tutorials/estimator/premade.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Запустить в Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ru/tutorials/estimator/premade.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Смотреть исходные файлы на GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ru/tutorials/estimator/premade.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Скачать ноутбук</a>
  </td>
</table>

Note: Вся информация в этом разделе переведена с помощью русскоговорящего Tensorflow сообщества на общественных началах. Поскольку этот перевод не является официальным, мы не гарантируем что он на 100% аккуратен и соответствует [официальной документации на английском языке](https://www.tensorflow.org/?hl=en). Если у вас есть предложение как исправить этот перевод, мы будем очень рады увидеть pull request в [tensorflow/docs](https://github.com/tensorflow/docs) репозиторий GitHub. Если вы хотите помочь сделать документацию по Tensorflow лучше (сделать сам перевод или проверить перевод подготовленный кем-то другим), напишите нам на [docs-ru@tensorflow.org list](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ru).

Это руководство покажет вам как решить проблему классификации цветов ириса в TensorFlow с помощью оценщиков. Оценщик - это высокоуровневое представление полной модели TensorFlow, которое было разработано для простого масштабирования и асинхронного обучения. Подробнее см.[Оценщики](https://www.tensorflow.org/guide/estimator).

Обратите внимание, что в TensorFlow 2.0 [Keras API](https://www.tensorflow.org/guide/keras) может выполнять многие из этих задач, и считается, что Keras API более простой для изучения. Если вы только начинаете свое обучение, мы рекомендуем вам начать с Keras. Для получения дополнительной информации о доступных API высокого уровня в TensorFlow 2.0 см. [Стандартизация для Keras](https://medium.com/tensorflow/standardizing-on-keras-guidance-on-high-level-apis-in-tensorflow-2-0-bad2b04c819a).

## Импорт

Для начала вы импортируете TensorFlow и ряд необходимых библиотек.

In [ ]:
import tensorflow as tf

import pandas as pd

## Датасет

Пример программы в этом документе создает и тестирует модель, которая классифицирует цветы ириса на три разных вида в зависимости от размера их [чашелистиков](https://en.wikipedia.org/wiki/Sepal) и [лепестков](https://en.wikipedia.org/wiki/Petal).


Вы обучите модель, используя набор данных Iris. Набор данных Iris содержит четыре признака и одну
[метку](https://developers.google.com/machine-learning/glossary/#label).
Четыре признака определяют следующие ботанические характеристики цветка ириса:

* длина чашелистика
* ширина чашелистика
* длина лепестка
* ширина лепестка

Основываясь на этой информации, вы можете определить несколько полезных констант для анализа данных:


In [ ]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

Теперь загрузите и проанализируйте датасет Iris с помощью Keras и Pandas. Обратите внимание, что вы сохраняете отдельные датасеты для обучения и тестирования.

In [ ]:
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)

Вы можете проверить данные, чтобы увидеть, что у вас есть четыре столбца признаков с типа float и одна метка типа int32.

In [ ]:
train.head()

Для каждого набора данных отделите метки.

In [ ]:
train_y = train.pop('Species')
test_y = test.pop('Species')

# Столбец метки теперь удален из признаков.
train.head()

## Введение в программирование с помощью оценщиков

Теперь, когда у вас есть данные, вы можете создать модель с помощью оценщика TensorFlow. Оценщик - это любой класс, производный от `tf.estimator.Estimator`. TensorFlow предоставляет коллекцию `tf.estimator`(например, `LinearRegressor`) для реализации общих алгоритмов машинного обучения. Кроме того, вы можете написать свой [собственный оценщик](https://www.tensorflow.org/guide/custom_estimators).
Для начала мы рекомендуем использовать готовые оценщики.

Чтобы написать программу TensorFlow на основе готовых оценщиков, необходимо выполнить
следующие шаги:

* Создайте одну или несколько функций ввода.
* Определите столбцы признаков модели.
* Создайте оценщик, указав признаки и различные гиперпараметры.
* Вызовите один или несколько методов объекта Estimator, передав соответствующую входную функцию как источник данных.

Давайте посмотрим, как эти задачи реализованы для классификации Iris.

## Создание функций ввода

Вы должны создать функции ввода для предоставления данных для обучения, оценки и прогноза.

**функция ввода** - это функция, возвращающая объект `tf.data.Dataset`, который выводит следующий двухэлементный кортеж:

* [`features`](https://developers.google.com/machine-learning/glossary/#feature) - словарь Python, в котором:
     * Каждый ключ - это имя признака.
     * Каждое значение представляет собой массив, содержащий все значения этого признака.
* `label` - Массив, содержащий значения [меток](https://developers.google.com/machine-learning/glossary/#label) для каждого примера.

Чтобы продемонстрировать формат функции ввода, вот простой пример реализации:

In [ ]:
def input_evaluation_set():
    features = {'SepalLength': np.array([6.4, 5.0]),
                'SepalWidth':  np.array([2.8, 2.3]),
                'PetalLength': np.array([5.6, 3.3]),
                'PetalWidth':  np.array([2.2, 1.0])}
    labels = np.array([2, 1])
    return features, labels

Ваша функция ввода может генерировать словарь `features` и список `label` любым способом, который вы предпочитаете. Однако мы рекомендуем использовать [Dataset API] TensorFlow(https://www.tensorflow.org/guide/datasets), который может работать с различными форматами входных данных.

`Dataset API` может работать с множеством распространенных случаев. Например, используя `Dataset API`, вы можете легко параллельно читать записи из большой коллекции файлов, объединив их в единый поток.

Для простоты в этом примере вы будете загружать данные с помощью [pandas](https://pandas.pydata.org/) и создадите конвейер ввода в памяти:

In [ ]:
def input_fn(features, labels, training=True, batch_size=256):
    """Функция ввода для обучения или оценки"""
    # Преобразуем входные данные в Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Перемешаем и повторим данные, если мы в режиме обучения.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)


## Определение столбцов признаков

[**столбец функций**](https://developers.google.com/machine-learning/glossary/#feature_columns)
- объект, описывающий, как модель должна использовать необработанные входные данные из словаря функций. Когда вы строите модель оценщика, вы передаете ей список столбцов признаков, которые описывают каждый признак, который должна использовать модель.
Модуль `tf.feature_column` предлагает множество опций для представления данных в модели.

Для Iris 4 необработанных признака представляют собой числовые значения, поэтому мы создадим список столбцов признаков, чтобы сообщить модели Оценщика, что каждый из четырех признаков должен быть представлен в виде float32. Следовательно, код для создания столбца признаков будет таким:

In [ ]:
# Столбцы признаков описывают, как использовать входные данные.
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

Столбцы признаков могут быть гораздо сложнее, чем те, которые мы показываем здесь. Вы можете узнать больше о столбцах признаков в [этом руководстве](https://www.tensorflow.org/guide/feature_columns).

Теперь, когда у вас есть описание того, как модель должна представлять необработанные
признаки, вы можете построить оценщик.

## Инициализация оценщика

Проблема распознования цветов ириса - это классическая проблема классификации. К счастью, TensorFlow
предоставляет несколько готовых оценщиков классификации, в том числе:

* `tf.estimator.DNNClassifier` для моделей глубокого обучения, которые выполняют мультиклассовую классификацию.
* `tf.estimator.DNNLinearCombinedClassifier` для больших и глубоких моделей.
* `tf.estimator.LinearClassifier` для классификаторов на основе линейных моделей.

Для распознования цветов ириса лучшим выбором кажется `tf.estimator.DNNClassifier`.
Вот пример создания этого эстиматора:

In [ ]:
# Создаем DNN с 2-мя скрытыми слоями с 30 и 10 скрытыми узлами в каждом.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Два скрытых слоя по 30 и 10 узлов соответственно.
    hidden_units=[30, 10],
    # Модель должна выбирать между 3 классами.
    n_classes=3)

## Обучение, оценка и прогноз

Теперь, когда у вас есть объект оценщика, вы можете вызывать методы, чтобы сделать следующее:

* Обучить модель.
* Оценить обученную модель.
* Использовать обученную модель, чтобы делать прогнозы.

### Обучение модели

Обучите модель, вызвав метод `train` эстиматора следующим образом:

In [ ]:
# обучение.
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=5000)

Обратите внимание, что вы оборачиваете вызов `input_fn` в [`lambda функцию`](https://docs.python.org/3/tutorial/controlflow.html) для передачи аргументов в функцию ввода. Аргумент `steps` указывает методу прекратить обучение после ряда шагов обучения.

### Оценка обученной модели

Теперь, когда модель обучена, вы можете получить некоторую статистику ее эффективности. Следующий блок кода оценивает точность обученной модели на тестовых данных:

In [ ]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(test, test_y, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

В отличие от вызова метода `train`, вы не передаете аргумент `steps` для оценки. `input_fn` для выполняется только для одной [эпохи](https://developers.google.com/machine-learning/glossary/#epoch).

Словарь `eval_result` содержит `average_loss`(средние потери на выборку), `loss`(средние потери на мини-пакет) и значение `global_step` оценщика(количество пройденных шагов обучения).

### Создание прогнозов (вывод) на основе обученной модели

Теперь у вас есть обученная модель, которая дает хорошие результаты оценки.
Вы можете использовать обученную модель, чтобы предсказать вид цветка ириса на основании некоторых немаркированных признаков. Как и в случае с обучением и оценкой, вы делаете прогнозы с использованием единственного вызова функции:

In [ ]:
# Создание прогнозов на основе модели
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
}

def input_fn(features, batch_size=256):
    """Функция ввода для прогноза."""
    # Преобразовываем входные данные в набор данных без меток.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

predictions = classifier.predict(
    input_fn=lambda: input_fn(predict_x))

Метод `predict` возвращает итератор Python, который возвращает словарь результатов прогноза для каждого примера. Следующий код выводит несколько прогнозов и их вероятностей:

In [ ]:
for pred_dict, expec in zip(predictions, expected):
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%), expected "{}"'.format(
        SPECIES[class_id], 100 * probability, expec))